<a href="https://colab.research.google.com/github/haku-noir/werewolf/blob/develop/werewolf_train_gpt_including_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DO_TRAIN = False

## ファイルパスの設定


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

DATA_DIR = "/content/drive/MyDrive/werewolf"

OUTPUT_DIR = os.path.join(DATA_DIR, "output")

GPT_TRAIN_TXT_PATH = os.path.join(DATA_DIR, "werewolf_io_messages_including_user.txt")
GPT_MODEL_DIR = os.path.join(OUTPUT_DIR, "model_werewolf_including_user")

## ライブラリのインストール

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece datasets evaluate

In [ ]:
!git clone https://github.com/huggingface/transformers

## ファインチューニング

In [ ]:
if DO_TRAIN:
    !python3 ./transformers/examples/pytorch/language-modeling/run_clm.py \
        --model_name_or_path=rinna/japanese-gpt2-medium \
        --train_file=$GPT_TRAIN_TXT_PATH \
        --validation_file=$GPT_TRAIN_TXT_PATH \
        --do_train \
        --do_eval \
        --num_train_epochs=3 \
        --save_steps=5000 \
        --save_total_limit=3 \
        --per_device_train_batch_size=1 \
        --per_device_eval_batch_size=1 \
        --output_dir=$GPT_MODEL_DIR \
        --use_fast_tokenizer=False \
        --overwrite_output_dir

## 文章の生成

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
tokenizer.do_lower_case = True
model = AutoModelForCausalLM.from_pretrained(GPT_MODEL_DIR)
model.to(device)
model.eval()

In [ ]:
def generate_message_including_user(input_user, input_text, output_user, num_gen=10):
    text = "<s>" + input_user + "[SEP]" + input_text + "[SEP]" + output_user + "[SEP]"
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    out = model.generate(input_ids, do_sample=True, top_p=0.95, top_k=40, 
                         num_return_sequences=num_gen, max_length=256, bad_words_ids=[[1], [5]])
    print('入力文')
    print(input_text)
    print('生成文')
    for sent in tokenizer.batch_decode(out):
        sent = sent.split('[SEP]</s>')[1]
        sent = sent.replace('</s>', '')
        sent = sent.replace('"', '')
        sent = sent.replace("'", "")
        sent = sent.replace(" ", "")
        print(sent)

In [ ]:
input_user = "楽天家 ゲルト"
input_text = "ふぁーあ……ねむいな……寝てていい？"
output_user = "少年 ペーター"
generate_message_including_user(input_user, input_text, output_user)